In [ ]:
import numpy as np
import pandas as pd

import SeasonData as sdata

In [ ]:
# Generating play-by-play data for year 2022
data = sdata.NHLData()  # Object creation
data.fetch_regular_season(year=2022)  # Fetching data
data_rs = data.regular_season  # Get data in variable

In [ ]:
def additional_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add additional features to the dataframe"""

    # Add shot distance

    # Define the Euclidian distance function
    dist_euclidian = lambda x1, x2: np.round(np.linalg.norm(np.array(x1 - x2)), decimals=1)

    # Add shot distance based on the ice coordinates
    df['shotDistance'] = df.apply(
        lambda x: dist_euclidian(x['iceCoord'], np.array([-89, 0])) if x['iceCoord'][0] <= 0
        else dist_euclidian(x['iceCoord'], np.array([89, 0])), axis=1)
    return df

In [ ]:
def minutes_to_seconds(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Convert dataframe column from 'minutes:seconds' to 'seconds' and add the number of period
    """
    # Split columns into 'minutes' and 'seconds' and 'number of period' as integer
    df['minutes'] = df[column].str.split(':').str[0].astype(int)
    df['seconds'] = df[column].str.split(':').str[1].astype(int)
    df['numberPeriod'] = df['currentPeriod'].str.split("/").str[0].astype(int)

    # Total in seconds
    df[column] = df['minutes'] * 60 + df['seconds'] + 20 * 60 * (df['numberPeriod'] - 1)

    #Drop columns
    df.drop(['minutes', 'seconds', 'numberPeriod'], axis=1, inplace=True)

    return df

In [ ]:
def extract_players(game_nhl: dict) -> pd.DataFrame:
    """Extract the play-by-play data from the game dictionary"""

    # Extract player data
    df_players = pd.DataFrame(game_nhl['rosterSpots'])[['playerId', 'firstName', 'lastName']]

    # Keep the default name for each player (first and last name)
    df_players['firstName'] = df_players['firstName'].apply(lambda x: x['default'])
    df_players['lastName'] = df_players['lastName'].apply(lambda x: x['default'])
    return df_players

In [ ]:
def extract_teams(game_nhl: dict) -> pd.DataFrame:
    """Extract 'home' and 'away' teams"""
    home_team = {'teamId': game_nhl['homeTeam']['id'], 'teamName': game_nhl['homeTeam']['name']['default'],
                 'teamSide': 'home'}
    away_team = {'teamId': game_nhl['awayTeam']['id'], 'teamName': game_nhl['awayTeam']['name']['default'],
                 'teamSide': 'away'}
    return pd.DataFrame([home_team, away_team])

In [ ]:
def process_period_data(df: pd.DataFrame) -> pd.DataFrame:
    """Decompose periodDescriptor and return processed dataframe."""
    
    # Split 'periodDescriptor' into 'periodType', 'number', and 'maxRegulationPeriods'
    df_period = pd.DataFrame(df['periodDescriptor'].tolist())

    # Convert 'number' and 'maxRegulationPeriods' columns as strings
    df_period[['number', 'maxRegulationPeriods']] = df_period[['number', 'maxRegulationPeriods']].astype(str)
    return df_period

In [ ]:
def process_event_details(df: pd.DataFrame, df_players: pd.DataFrame) -> pd.DataFrame:
    """Process event details and merge with player information."""
    df_details = pd.DataFrame(df['details'].tolist())

    # Combine x and y coordinates into a tuple
    df_details['iceCoord'] = df_details[['xCoord', 'yCoord']].apply(tuple, axis=1)

    # Merge 'shooting' and 'scoring' player, to keep only one column
    df_details['shootingPlayerId'] = df_details['shootingPlayerId'].fillna(0) + df_details['scoringPlayerId'].fillna(0)

    # Convert 'shootingPlayerId' and 'goalieInNetId' as integer
    df_details['shootingPlayerId'] = df_details['shootingPlayerId'].astype(int)
    df_details['goalieInNetId'] = df_details['goalieInNetId'].astype('Int64')  # Int64: handling NaN values

    # Add the shooter names by merging IDs
    df_details = pd.merge(df_players, df_details, left_on='playerId', right_on='shootingPlayerId', how='right').drop(
        columns=['playerId'])

    # Keep only full name
    df_details['shootingPlayer'] = df_details['firstName'] + ' ' + df_details['lastName']
    df_details.drop(['firstName', 'lastName'], axis=1, inplace=True)

    # Add the goalies names by merging IDs 
    df_details = pd.merge(df_players, df_details, left_on='playerId', right_on='goalieInNetId', how='right').drop(
        columns=['playerId'])

    # Keep only full name
    df_details['goaliePlayer'] = df_details['firstName'] + ' ' + df_details['lastName']
    df_details.drop(['firstName', 'lastName'], axis=1, inplace=True)

    return df_details

In [ ]:
def calculate_empty_goal_net(df: pd.DataFrame) -> pd.Series:
    """Determine if the goal net is empty"""
    return df.apply(lambda x: x['situationCode'][3] if x['teamSide'] == 'away' else x['situationCode'][0], axis=1).map(
        {'0': True, '1': False})

In [ ]:
def determine_goal_advantage(df: pd.DataFrame) -> pd.Series:
    """Determine if the event team is in advantage, disadvantage, or neutral situation"""
    return df.apply(lambda x: "Advantage" if (int(x['situationCode'][1]) > int(x['situationCode'][2]) and x[
        'teamSide'] == 'away') or (int(x['situationCode'][2]) > int(x['situationCode'][1]) and x[
        'teamSide'] == 'home')
    else "Disadvantage" if (int(x['situationCode'][1]) < int(x['situationCode'][2]) and x['teamSide'] == 'away') or
                           (int(x['situationCode'][2]) < int(x['situationCode'][1]) and x['teamSide'] == 'home')
    else 'Neutral', axis=1)

In [ ]:
def convert_event_to_dataframe(game_nhl: dict) -> pd.DataFrame:
    """
    Convert NHL game event data into a clean dataframe  
    :param game_nhl: Dictionary containing the data of the NHL game
    :return: A Pandas DataFrame containing filtered data
    """
    # Extract play-by-play data
    df_pbp = pd.DataFrame(game_nhl['plays'])

    # Extract player and team data
    df_players = extract_players(game_nhl)
    df_teams = extract_teams(game_nhl)

    # Create a new dataframe for the event data
    clean_df = pd.DataFrame(df_pbp[['periodDescriptor', 'timeInPeriod', 'situationCode', 'typeDescKey', 'details']])

    # Insert the game ID as the first column
    clean_df.insert(0, 'idGame', game_nhl['id'])

    # Filter to keep only events of type 'shot-on-goal' or 'goal'
    clean_df = clean_df[(clean_df['typeDescKey'] == 'shot-on-goal') | (clean_df['typeDescKey'] == 'goal')].reset_index(
        drop=True)

    # Create a dataframe to decompose the period descriptor fields
    df_period = process_period_data(clean_df)
    clean_df.drop('periodDescriptor', axis=1, inplace=True)

    # Add 'periodType' and 'currentPeriod' columns to the new dataframe
    clean_df.insert(1, 'periodType', df_period['periodType'])
    clean_df.insert(2, 'currentPeriod', df_period['number'] + '/' + df_period['maxRegulationPeriods'])

    # Parse the game's start time to UTC
    start_time = pd.to_datetime(game_nhl['startTimeUTC'])

    # Convert time in the period to seconds and add it to the game start time
    clean_df = minutes_to_seconds(clean_df, 'timeInPeriod')
    clean_df['timeInPeriod'] = start_time + pd.to_timedelta(clean_df['timeInPeriod'], unit='s')

    # Process event details and merge player information
    df_details = process_event_details(clean_df, df_players)
    clean_df.drop('details', axis=1, inplace=True)

    # Add team data by merging IDs
    df_details = pd.merge(df_teams, df_details, left_on='teamId', right_on='eventOwnerTeamId', how='right')

    # Add the extracted data to the new dataframe
    clean_df['iceCoord'] = df_details['iceCoord']
    clean_df['shootingPlayer'] = df_details['shootingPlayer']
    clean_df['goaliePlayer'] = df_details['goaliePlayer']
    clean_df['shotType'] = df_details['shotType']
    clean_df.insert(5, 'eventOwnerTeam', df_details['teamName'])
    clean_df['teamSide'] = df_details['teamSide']

    # Calculate emptyGoalNet and goal advantage
    clean_df['emptyGoalNet'] = calculate_empty_goal_net(clean_df)
    clean_df['isGoalAdvantage'] = determine_goal_advantage(clean_df)

    # Add shot distance
    clean_df = additional_features(clean_df)

    return clean_df

In [ ]:
# Test one game (avoid to test on all data)

new_data = sdata.NHLData()
new_data.fetch_regular_season(year=2022)
data_2022 = data.regular_season[2022]
game_one_2022 = data_2022[0]
df_game_one_2022 = convert_event_to_dataframe(game_one_2022)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_game_one_2022)

In [ ]:
#Test on all regular season

data_rs_clean = {}  # Store all clean data

#Loop on all years in regular season
for year in data_rs:
    current_year = data_rs[year]  #Get year
    sdata_year = len(current_year)  #Get the number of games in the year
    data_rs_clean[year] = []  #Create year key to store all games

    #Loop on all game in the year
    for game in range(sdata_year):
        current_year[game] = convert_event_to_dataframe(game_nhl=current_year[game])  #Processing game data
        data_rs_clean[year].append(current_year[game])  #Append the clean game data to the year key

display(data_rs_clean[2022][0])  #Display the first game of 2022